In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *


import os
from sklearn.model_selection import KFold


In [2]:
train = pd.read_pickle('e:/kaggle/09504/train.pkl')
test = pd.read_pickle('e:/kaggle/09504/test.pkl')
y = pd.read_pickle('e:/kaggle/09504/y.pkl')

In [3]:
from BayDS.lib.training import *
from category_encoders.cat_boost import CatBoostEncoder

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [4]:
test.columns

Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       'Transaction_freq_2d_past', 'Transaction_freq_3d_past',
       'Transaction_freq_7d_past', 'Transaction_freq_30d_past',
       'Transaction_freq_5_past', 'Transaction_freq_5_center',
       'Transaction_freq_10_past', 'Transaction_freq_10_center',
       'Transaction_freq_100_past', 'Transaction_freq_100_center'],
      dtype='object', length=698)

In [5]:
import yaml
cat_col = yaml.load(open(r'e:/kaggle/09504/categorical_columns.yaml','r'),Loader=yaml.FullLoader) 
num_col = yaml.load(open(r'e:/kaggle/09504/numerical_columns.yaml','r'),Loader=yaml.FullLoader) 


In [6]:
undef_cols = []
for col in test.columns:
    if col not in cat_col and col not in num_col:
        print (col)
#         undef_cols.append(col)
# yaml.dump( undef_cols,open('e:/kaggle/09504/undef_cols.yaml','w') )
        

In [8]:
X = train.values

In [9]:
y = y.values

In [13]:
X[y==1].shape

(20663, 698)

In [14]:
q = np.array([1,2,3,4,5])
q==3

array([False, False,  True, False, False])

In [7]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47,
          }

n_fold = 5
folds = KFold(n_splits=n_fold)
train_options = {
    "model_type": 'lgb',
    "params": params,
    "eval_metric": 'auc',
    'early_stopping_rounds': 200,
    'n_estimators': 10000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold}

result_timefreq = train_model_classification_vb(X=train, X_test=test, y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'],
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'],
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=None,
                                             train_1_sample_coef=None, train_0_sample_coef=0.2,
#                                              categorial_columns=cat_col, categorial_encoder=CatBoostEncoder
                                               )

# In[50]:


pd.DataFrame(result_timefreq['oof'], columns=['isFraud'], index=train.index).to_csv(
    'oof_all_data_best_lgb_timefreq.csv')
pd.DataFrame(result_timefreq['prediction'], columns=['isFraud'], index=test.index).to_csv(
    'prediction_all_data_best_lgb_timefreq.csv')

sample_submission = pd.read_csv(data_path + 'sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_timefreq['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1  # *0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv('lgb_timefreq.csv')


Fold 1 started at Sat Sep 28 21:08:26 2019
Fold 2 started at Sat Sep 28 21:08:35 2019
Fold 3 started at Sat Sep 28 21:08:44 2019
Fold 4 started at Sat Sep 28 21:08:54 2019
Fold 5 started at Sat Sep 28 21:09:02 2019


ValueError: Found input variables with inconsistent numbers of samples: [107765, 472432]